<a href="https://colab.research.google.com/github/smithkanzariya/Fake_News_Classifier-Using-Bidirectional-LSTM/blob/main/FakeNewsClassifierUsingBidirectionalLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
#Loading Dataset
import pandas as pd
Dataset  = pd.read_csv("/content/drive/My Drive/train.csv")
Dataset.head(10)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
6,6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,NaN,Ever wonder how Britain’s most iconic pop pian...,1
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
8,8,Excerpts From a Draft Script for Donald Trump’...,NaN,Donald J. Trump is scheduled to make a highly ...,0
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0


Dealing with Nan Values

In [54]:
Dataset.shape

(20800, 5)

In [55]:
Dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [56]:
#Drop NAN Values
Dataset = Dataset.dropna()

In [57]:
Dataset.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [58]:
Dataset.shape

(18285, 5)

Deviding the Dataset into Dependent and Independent

In [59]:
#Get the independent Feature
X = Dataset.drop("label", axis = 1)
X.shape
Y = Dataset["label"]
Y.shape

(18285,)

In [60]:
Y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [61]:
X.shape


(18285, 4)

In [62]:
#Importing the tensorflow library
import tensorflow as tf 

In [63]:
tf.__version__

'2.8.0'

In [64]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import  Bidirectional

In [65]:
#Vocabulary size
voc_size = 5000

Onehot Representaion

In [66]:
messages = X.copy()
messages.reset_index(inplace = True)
messages["title"][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [67]:
import nltk
import re
from nltk.corpus import stopwords

In [68]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [69]:
#Data Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
  #print(i)
  review = re.sub("[^a-zA-Z]", " ", messages["title"][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if word not in stopwords.words("english")]
  review = " ".join(review)
  corpus.append(review)

In [70]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [71]:
onehot_repr = [one_hot(words, voc_size) for words in corpus]
onehot_repr

[[4009, 2522, 3905, 3348, 2085, 2461, 1807, 3879, 2080, 109],
 [3548, 521, 3302, 2478, 1013, 3415, 3223],
 [3045, 3856, 2513, 926],
 [4798, 2273, 4258, 659, 4347, 4352],
 [2650, 1013, 2232, 1218, 347, 719, 1013, 4737, 847, 779],
 [447,
  3306,
  3132,
  14,
  187,
  165,
  4781,
  3294,
  2699,
  4325,
  2621,
  2565,
  374,
  1840,
  3223],
 [1633, 1415, 3015, 1439, 2328, 788, 1706, 1169, 4136, 4022, 2661],
 [805, 1555, 1347, 4577, 4314, 1134, 165, 4446, 4136, 4022, 2661],
 [1101, 1818, 4670, 4126, 1066, 1926, 3889, 1560, 165, 1794],
 [1604, 4620, 4621, 2197, 3690, 1788, 1030, 2497],
 [2279, 4953, 846, 2906, 4345, 34, 583, 4641, 3951, 578, 1908],
 [659, 1523, 2085, 1926, 165, 4314],
 [3365, 3367, 3086, 3197, 1971, 2994, 2331, 2953, 2026],
 [1332, 4241, 4595, 2916, 3846, 1677, 65, 4136, 4022, 2661],
 [191, 4790, 3347, 3176, 4861, 4136, 4022, 2661],
 [4991, 4907, 4184, 1244, 3989, 2430, 2590, 1005, 130, 4311],
 [2770, 2939, 521],
 [1493, 4711, 1839, 2946, 165, 4036, 164, 3223],
 [4157, 

In [72]:
longest = max(onehot_repr, key = len)
print(len(longest))

47


Embedding Rpresentation

In [73]:
sent_lenght = 20
embedded_docs = pad_sequences(onehot_repr, padding="pre", maxlen= sent_lenght)
print(embedded_docs)

[[   0    0    0 ... 3879 2080  109]
 [   0    0    0 ... 1013 3415 3223]
 [   0    0    0 ... 3856 2513  926]
 ...
 [   0    0    0 ... 4136 4022 2661]
 [   0    0    0 ... 3419 2170 4387]
 [   0    0    0 ...   41 3945 4765]]


In [74]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4009,
       2522, 3905, 3348, 2085, 2461, 1807, 3879, 2080,  109], dtype=int32)

Build The Model

In [75]:
from tensorflow.keras.layers import Dropout
#Creating model
embedding_vector_feature = 40
model1 = Sequential()
model1.add(Embedding(voc_size, embedding_vector_feature, input_length = sent_lenght))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dense(1, activation= "sigmoid"))
model1.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])
print(model1.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional_1 (Bidirectio  (None, 200)              112800    
 nal)                                                            
                                                                 
 dense_1 (Dense)             (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [76]:
len(embedded_docs), Y.shape

(18285, (18285,))

In [77]:
import numpy as np
X_final = np.array(embedded_docs)
Y_final = np.array(Y)

In [78]:
X_final.shape, Y_final.shape

((18285, 20), (18285,))

In [79]:
#Splitng the data into train and test.
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_final, Y_final, test_size = 0.33, random_state = 42)

In [81]:
X_train.shape, Y_train.shape

((12250, 20), (12250,))

Model Training

In [82]:
#Training Time
model1.fit(X_train, Y_train, validation_data = (X_test, Y_test), epochs = 10, batch_size = 64)

Epoch 1/10
192/192 [==============================] - 14s 56ms/step - loss: 0.2948 - accuracy: 0.8602 - val_loss: 0.2024 - val_accuracy: 0.9201
Epoch 2/10
192/192 [==============================] - 10s 50ms/step - loss: 0.1386 - accuracy: 0.9467 - val_loss: 0.1975 - val_accuracy: 0.9193
Epoch 3/10
192/192 [==============================] - 10s 50ms/step - loss: 0.0953 - accuracy: 0.9668 - val_loss: 0.2170 - val_accuracy: 0.9170
Epoch 4/10
192/192 [==============================] - 10s 50ms/step - loss: 0.0683 - accuracy: 0.9760 - val_loss: 0.2933 - val_accuracy: 0.9142
Epoch 5/10
192/192 [==============================] - 10s 51ms/step - loss: 0.0437 - accuracy: 0.9862 - val_loss: 0.3124 - val_accuracy: 0.9118
Epoch 6/10
192/192 [==============================] - 10s 50ms/step - loss: 0.0318 - accuracy: 0.9898 - val_loss: 0.3342 - val_accuracy: 0.9145
Epoch 7/10
192/192 [==============================] - 10s 50ms/step - loss: 0.0236 - accuracy: 0.9925 - val_loss: 0.3905 - val_accuracy:

Performence Metrics and Accuracy 

In [116]:

Y_pred1 = (model1.predict(X_test) > 0.5).astype("int32")

In [117]:
Y_pred1

array([[1],
       [0],
       [0],
       ...,
       [0],
       [1],
       [1]], dtype=int32)

In [118]:
len(Y_pred1)

6035

In [119]:
from sklearn.metrics import confusion_matrix

In [120]:
type(Y_test), type(Y_pred1)

(numpy.ndarray, numpy.ndarray)

In [121]:
confusion_matrix(Y_test, Y_pred1)

array([[3122,  297],
       [ 218, 2398]])

In [122]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, Y_pred1)

0.9146644573322287

In [123]:
from sklearn.metrics import  classification_report
print(classification_report(Y_test, Y_pred1))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      3419
           1       0.89      0.92      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.92      0.91      0.91      6035

